# Tarea 5

**Envío del PR inicial:** 15 de octubre

**Aceptación del PR:** 21 de octubre

NOTA: Esta tarea debe entregarse en equipo.

## 1

Llamemos $c_n$ el valor del parámetro $c$ donde ocurre la bifurcación de doblamiento de periodo para el mapeo $Q_c(x)=x^2+c$, donde la órbita de periodo $2^n$ nace. Como hemos visto en notebooks anteriores, tenemos que $c_0=1/4$ marca la aparición del atractor de periodo $2^0=1$, $c_1=-1/4$ corresponde a la aparición del atractor de periodo $2^1=2$, $c_2=-3/4$ a la aparición del atractor de periodo $2^2=4$, etc. 

A partir de estos valores y otros que calcularán (al menos deben encontrar $c_6$), definimos la secuencia: $\{f_0, f_1, f_2, \dots\}$, donde

\begin{equation}
f_n = \frac{c_n-c_{n+1}}{c_{n+1}-c_{n+2}} .
\end{equation}

La pregunta es, ¿a qué valor converge esta secuencia?, es decir, dar una estimación de $f_\infty$.


*Hint:* Para realizar este ejercicio deben calcular el atractor para varios valores de $c$, de tal manera que puedan aislar las órbitas de periodo $2^p$ y de ahí determinar varios valores $c_n$. Se requerir suficiente cuidado para obtener una buena aproximación de $c_n$. 
Una opción se basa en recordar/usar que las bifurcaciones de doblamiento de periodo ocurren cuando los puntos de la órbita de periodo $p$ se tornan en repulsores, es decir, $(Q_c^p)'(x)=-1$. Esta opción, entonces, involucra obtener los valores $c_n$ a partir de órbitas periódicas de periodo $2^n$ usando los polinomios $Q_c^{2^p}(x)$ y diferenciación automática.

In [90]:
"""
    ciclosestables!(xx, f, cc, nit, nout)

Esta función itera el mapeo `f`, de una variable, `nit+nout` veces, 
usando como condición inicial `x0=0`; los últimos `nout` iterados 
actualizan al vector `xx` que tiene longitud `nout`. `cc` es el valor
del parámetro del mapeo `f`. El mapeo `f` debe ser definido de 
tal manera que `f(x0, cc)` tenga sentido. La idea es que los últimos 
`nout` iterados reflejen los ciclos estables del mapeo `f`.
"""
function ciclosestables!(xx, f, cc, nit, nout)
    @assert (nit > 0) && (nout > 0)
    
    # Primeros nit iterados
    x0 = 0.0
    for it = 1:nit
        x0 = f(x0, cc)
    end
    
    # Se guardan los siguientes nout iterados
    for it = 1:nout
        x0 = f(x0, cc)
        @inbounds xx[it] = x0
    end
    
    nothing
end

ciclosestables!

In [2]:
include("DualNum.jl")

Main.DualNum

In [3]:
"""
    diag_bifurc(f, nit, nout, crange)

Itera el mapeo `f` `nit+nout` veces y regresa una matriz
cuya columna `i` tiene los últimos `nout` iterados del mapeo
para el valor del parámetro del mapeo `crange[i]`.

La función `f` debe ser definida de tal manera que `f(x0, c)` 
tenga sentido.
"""
function diag_bifurc(f, nit, nout, crange)
#     xx = Vector{Float64}(nout)
    ff = Array{Float64,2}(undef, nout, length(crange))
    
    for ic in eachindex(crange)
        c = crange[ic]
       ciclosestables!(view(ff, :, ic), f, c, nit, nout)
#         ff[:,ic] = xx
    end
    
    return ff
end

diag_bifurc

In [4]:
function Newton(f,Xo,iteraciones)
    for i in 1:iteraciones
        Xo = Xo - f(DualNum.dual(Xo)).x/f(DualNum.dual(Xo)).x´
    end
    return Xo
end

Newton (generic function with 1 method)

In [5]:
Qc(x,c) = x^2 + c

Qc (generic function with 1 method)

In [6]:
"""
    C(f,c,p)

Dada la función `f(x,c)`, encuentra el valor del parámetro en el
rango `c` donde el punto fijo atractor se vuelve repulsor, 
es decir, donde hay una bifurcación de periodo `p` a periodo `p+2`. 
"""
function C(f,c,p)
    
    A = diag_bifurc(f, 2000000, p, c)
    dc = 0
    r = 1
    g = DualNum.dual(A[1,r])
    
    while g.x´ > -1
        r = r+1
        g = DualNum.dual(A[1,r])      
        for j in 1:p
            g = f(g, c[r])
        end 
        #println(g.x´,c[r])
        if g.x´ < -1.0
            dc = c[r]
            break
        end
    end
    return dc
end

C

In [17]:
# sea el vector C_n
Cn = []
push!(Cn,0.25)
push!(Cn,C(Qc,-0.749:-1/2^18:-0.751,1))
push!(Cn,C(Qc,-1.24:-1/2^18:-1.26, 2))
push!(Cn,C(Qc,-1.365:-1/2^18:-1.370,4))
push!(Cn,C(Qc,-1.393:-1/2^18:-1.395,8))
push!(Cn,C(Qc,-1.395:-1/2^18:-1.4,16)) 
push!(Cn,C(Qc,-1.4:-1/2^18:-1.401,32))             #-1.4008287
push!(Cn,C(Qc,-1.401:-1/2^18:-1.42,64))            #1M

8-element Array{Any,1}:
  0.25              
 -0.7499994506835937
 -1.2499983215332031
 -1.3681013488769531
 -1.394049041748047 
 -1.3996348571777344
 -1.4008316040039062
 -1.4010877380371094

In [7]:
fn(cn,cn1,cn2) = (cn-cn1)/(cn1-cn2) 

fn (generic function with 1 method)

In [19]:
Fn = []

for i in 1:6
    push!(Fn,fn(Cn[i],Cn[i+1],Cn[i+2]))
end

In [22]:
Fn

6-element Array{Any,1}:
 2.000003417976469
 4.233582170542635
 4.551581056954741
 4.645282895168167
 4.667499681244839
 4.672346002618775

La sucesión $f_n$ para la función cuadrática converge a un valor cercano a $4.672346...$

## 2

Repitan el ejercicio anterior para el mapeo $S_c(x) = c \sin(x)$. 

- ¿Cómo se comparan los valores obtenidos de $f_n$? 

- ¿Qué interpretación le pueden dar a este resultado, en comparación del ejercicio anterior?

In [80]:
"""
    Cs(f,c,p)

Dada la función `f(x,c)`, encuentra el valor del parámetro en el
rango `c` donde el punto fijo atractor se vuelve repulsor, 
es decir, donde hay una bifurcación de periodo `p` a periodo `p+2`. 
"""
function Cs(f,c,p)
    
    A = diag_bifurc(f, 2000005, p, c)       #Se requirió de un número impar de iterados, debido a que con los pares
    dc = 0                                  #coincide que al cambiar de periodo, el punto final pertenece a una órbita del nuevo
    r = 1                                   #periodo, pero que no cumple con la condición de que a derivada es < -1 por la forma de 
    g = DualNum.dual(A[1,r])                #las órbitas y la función sin(x)
    
    while g.x´ > -1
        r = r+1
        g = DualNum.dual(A[1,r])      
        for j in 1:p
            g = f(g, c[r])
        end 
        #println(g, -c[r])
        if g.x´ < -1 
            dc = c[r]
            break
        end
    end
    return dc
end

Cs

In [87]:
Sc(x,c) = c*sin(x)
Cns = []
push!(Cns,1.0)
push!(Cns,Cs(Sc, 2.26:1/2^18:2.265, 1))
push!(Cns,Cs(Sc, 2.615:1/2^18:2.620, 2))
push!(Cns,Cs(Sc, 2.695:1/2^18:2.7,4))
push!(Cns,Cs(Sc, 2.712:1/2^18:2.715,8))
push!(Cns,Cs(Sc, 2.718:1/2^18:2.719,16))
push!(Cns,Cs(Sc, 2.719:1/2^18:2.7192,32))
push!(Cns,Cs(Sc, 2.7192:1/2^18:2.7193,64))

8-element Array{Any,1}:
 1.0               
 2.261827239990234 
 2.6177847290039065
 2.6974032592773436
 2.714830505371094 
 2.718293731689453 
 2.7190839233398436
 2.71926103515625  

In [88]:
Fsn = []
for i in 1:6
    push!(Fsn, fn(Cns[i],Cns[i+1],Cns[i+2]))
end

In [89]:
Fsn

6-element Array{Any,1}:
 3.544881843859078 
 4.470786986285645 
 4.568623742680114 
 5.032084100702723 
 4.382767543351822 
 4.4615411124129185

Se observa que la sucesión $f_n$ para la función $c\sin(x)$, también converge al mismo valor que la función cuadrática $x^2+c$.

Al existir una región de $c$ para ambas funciónes en las que el número de periodos llega a $2^{\infty}$ a demás de que el rango en el que cae la órbita es acotado, se habla de caos en tales funciones.

Pero se observa que el "camino" al caos se da, para almenos las funciones $c\sin(x)$ y $x^2+c$, de una manera "ordenada", en el sentido de lo que nos muestra la sucesión $f_n$, que es, las distancias entre las $c$ donde se bifurca cada rama de periodo $2^p$ obedecen una proporción de $\approx 4.672346...$

## 3

Como se ve en la Fig. 1 del diagrama de bifurcaciones de $Q_c$, $x=0$ pertenece a un ciclo de periodo $2^n$ para ciertos valores $C_n$ del parámetro. Dichos valores son *especiales*, ya que el hecho de que $x=0$ pertenezca a un ciclo de periodo $2^n$ define los llamados *ciclos superestable*, donde tenemos $(Q^{2^p}_{C_n})'(0)=0$.

- ¿A qué converge la secuencia $f_n$, definida ahora a partir de los valores $C_n$.

- De los $2^p$ puntos del ciclo de periodo $2^p$, es decir, $\{0, p_1, \dots p_{2^{n-1}}\,\}$ hay uno (distinto del 0) cuya distancia a 0 es la menor; a esa distancia la identificaremos como $d_n$. Calcular numéricamente a qué converge la secuencia $d_n/d_{n+1}$.

In [92]:
function Cz(f,c,p)
    
    A = diag_bifurc(f, 100000, p, c)
    dc = 0
    r = 1
    g = DualNum.dual(A[1,r])
    
    while g.x´ > 0.0
        r = r+1
        g = DualNum.dual(A[1,r])      
        for j in 1:p
            g = f(g, c[r])
        end 
        #println(g.x´,"  ",c[r])
        if g.x´ < 0.0
            dc = c[r]
            break
        end
    end
    return dc
    
    while g.x´ < 0.0
        r = r+1
        g = DualNum.dual(A[1,r])      
        for j in 1:p
            g = f(g, c[r])
        end 
        #println(g.x´,"  ",c[r])
        if g.x´ > 0.0
            dc = c[r]
            break
        end
    end
    return dc
end

Cz (generic function with 1 method)

In [93]:
Czn = []

push!(Czn, 0.0)
push!(Czn, -1.0)
push!(Czn, Cz(Qc, -1.30:-1/10^6:-1.32, 4))
push!(Czn, Cz(Qc, -1.38:-1/10^6:-1.385, 8))
push!(Czn, Cz(Qc, -1.396:-1/10^6:-1.398, 16))
push!(Czn, Cz(Qc, -1.40015:-1/10^6:-1.40035, 32))
push!(Czn, Cz(Qc, -1.4009:-1/10^8:-1.4010, 64))
push!(Czn, Cz(Qc, -1.4010:-1/10^8:-1.40125, 128))
push!(Czn, Cz(Qc, -1.40114:-1/10^8:-1.40115, 256))

9-element Array{Any,1}:
  0.0       
 -1.0       
 -1.310703  
 -1.381548  
 -1.396946  
 -1.400254  
 -1.40096197
 -1.40111381
 -1.40114633

In [94]:
Fzn = []

for i in 1:7
    push!(Fzn,fn(Czn[i],Czn[i+1],Czn[i+2]))
end

In [95]:
Fzn

7-element Array{Any,1}:
 3.218507706716704 
 4.385672947985034 
 4.600922197688008 
 4.6547762998789635
 4.672514372078756 
 4.662605374076825 
 4.669126691270171 

In [96]:
D = []
for i in 1:9

    A = diag_bifurc(Qc, 1000000, 2^(i-1), Czn[i])          #Se usarán los valores de c determinados en el inciso anterior.
    
    absA = []                    #Se tomarán los valores absolutos del vector A, para poder nombrar al más cercano a cero
                                 #como el mínimo de éste vector absA
    for l in 1:length(A)
        push!( absA , abs(A[l]))
    end
    
    min = minimum(absA)
    s = 0
    
    for r in 1:length(absA)      #Se guarda el ínidce tal que el la entrada de A es la más cercana a cero.
        if absA[r] == min
            s = r
        end
    end


    D1 = []
    
    if i == 1
        push!(D1,0.0)
    end
    
    for m in 1:length(A)        #Se realiza la resta de los demás puntos fijos de A, con el más cercano a cero y se guardan en D1
        if m != s
            push!(D1, abs(min-A[m]))
        end
    end
    
    push!(D, minimum(D1))       #Por último en el vector D, se guarda el menor de las restas anteriores.
    
end

In [97]:
D

9-element Array{Any,1}:
 0.0                  
 1.0                  
 0.4072387322583946   
 0.16342849226476241  
 0.06536302595334376  
 0.026140533748405748 
 0.010436909263090488 
 0.004170337459091122 
 0.0016660259442164982

In [98]:
dn = []
for i in 1:8
    push!(dn, D[i]/D[i+1])
end
dn

8-element Array{Any,1}:
 0.0               
 2.4555621083838757
 2.4918465967283554
 2.500320171551084 
 2.500447258745438 
 2.504624030875711 
 2.5026534100588336
 2.5031647757756597

Nuevamente, se observa la misma proporción $4.672346...$ entre las $C's$ tales que la órbita es super estable. 

Y se tiene una nueva constante $d \approx 2.503...$ que representa también una proporción entre los puntos mas cercanos a $x=0$ en dos órbita superestables consecutivas. Éste número representa una proporción en el ancho de las bifurcaciones.